## Add source code folder to PATH

In [17]:
import os
import sys

sys.path.append(os.path.abspath("../labs"))

## Django setup

In [18]:
from django.core.wsgi import get_wsgi_application

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings")
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "True")

os.environ.setdefault("LOCAL_LLM_HOST", "http://localhost:11434")
os.environ.setdefault("DATABASE_HOST", "localhost")

application = get_wsgi_application()

In [19]:
EMBEDDER_MODEL_NAME = "nomic-embed-text:latest"
LLM_MODEL_NAME = "starcoder2:15b-instruct"

REPO = "/Users/jsilva/Documents/revent/revent-api"
ISSUE = "Add created_at and updated_at field to User model."

## Embeddings

In [20]:
from embeddings.embedder import Embedder
from embeddings.ollama import OllamaEmbedder

embedder = Embedder(OllamaEmbedder, EMBEDDER_MODEL_NAME)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/jsilva/Library/Caches/pypoetry/virtualenvs/labs-_xSB_XeG-py3.12/lib/python3.12/site-packages/certifi/cacert.pem'


## Vectorize

In [21]:
from embeddings.vectorizers.base import Vectorizer
from embeddings.vectorizers.chunk_vectorizer import ChunkVectorizer

Vectorizer(ChunkVectorizer, embedder).vectorize_to_database(None, repo_destination=REPO)

DEBUG:embeddings.vectorizers.chunk_vectorizer:Loading and splitting all documents into chunks.
DEBUG:embeddings.vectorizers.chunk_vectorizer:Embedding all repo documents.
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=11434 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1463d55e0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Tue, 26 Nov 2024 11:34:58 GMT'), (b'Transfer-Encoding', b'chunked')])
INFO:httpx:H

## Embeddings:lookup

In [22]:
embeddings_results = embedder.retrieve_embeddings(ISSUE, REPO)

similar_embeddings = [(embedding.repository, embedding.file_path, embedding.text) for embedding in embeddings_results]

DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Tue, 26 Nov 2024 11:34:59 GMT'), (b'Transfer-Encoding', b'chunked')])
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete


In [23]:
similar_embeddings

[('/Users/jsilva/Documents/revent/revent-api',
  '/Users/jsilva/Documents/revent/revent-api/photo/migrations/0005_add_timestamps_to_user.py',
  "from django.db import migrations, models\n\nclass Migration(migrations.Migration):\n\n    dependencies = [\n        ('photo', '0004_alter_contest_prize'),\n    ]\n\n    operations = [\n        migrations.AddField(\n            model_name='user',\n            name='created_at',\n            field=models.DateTimeField(auto_now_add=True),\n        ),\n        migrations.AddField(\n            model_name='user',\n            name='updated_at',\n            field=models.DateTimeField(auto_now=True),\n        ),\n    ]"),
 ('/Users/jsilva/Documents/revent/revent-api',
  '/Users/jsilva/Documents/revent/revent-api/pytest.ini',
  '[pytest]\nDJANGO_SETTINGS_MODULE = config.settings'),
 ('/Users/jsilva/Documents/revent/revent-api',
  '/Users/jsilva/Documents/revent/revent-api/Dockerfile',
  'FROM python:3.10.13 AS base\n\n# Configure volume/workdir\nRUN 

## Prompt

In [24]:
from llm import get_prompt, prepare_context

prompt = get_prompt(ISSUE)
prepared_context = prepare_context(similar_embeddings, prompt)

## LLM

In [25]:
from litellm_service.llm_requester import Requester
from litellm_service.ollama import OllamaRequester
from llm import validate_llm_response

requester = Requester(OllamaRequester, model=LLM_MODEL_NAME)
llm_response = requester.completion_without_proxy(prepared_context)
redo, redo_reason = validate_llm_response(llm_response)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/jsilva/Library/Caches/pypoetry/virtualenvs/labs-_xSB_XeG-py3.12/lib/python3.12/site-packages/certifi/cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=11434 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1464b8710>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Tue, 26 Nov 2

In [34]:
import json
from pprint import pprint

print(EMBEDDER_MODEL_NAME)
print(LLM_MODEL_NAME)

print(REPO)
print(ISSUE)

pprint(json.loads(llm_response[1]["choices"][0]["message"]["content"].replace("\n", " ")))
# pprint(llm_response[1]["choices"][0]["message"]["content"].replace("\\n", " "))

nomic-embed-text:latest
starcoder2:15b-instruct
/Users/jsilva/Documents/revent/revent-api
Add created_at and updated_at field to User model.
{'steps': [{'content': '', 'path': 'project_name/models.py', 'type': 'create'},
           {'content': '',
            'path': 'project_name/tests/test_models.py',
            'type': 'create'}]}
